In [1]:
from math import inf
X = START = HORIZONTAL = 0
Y = END = VERTICAL = 1
DIRECTION = DIAGONAL = 2

In [2]:
def print_data(data, num, max_nums, min_nums):
    num = min(num, len(data))
    print("First {} (from a total of {}) line segments:".format(num, len(data)))
    for segment in data[:num]:
        desc = "({}, {})\t->\t({}, {}):\t".format(segment[START][X], segment[START][Y], segment[END][X], segment[END][Y])
        if segment[DIRECTION] == HORIZONTAL:
            desc += "HORIZONTAL of size\t" + str(abs(segment[START][X] - segment[END][X]))
        elif segment[DIRECTION] == VERTICAL:
            desc += "VERTICAL of size\t" + str(abs(segment[START][Y] - segment[END][Y]))
        else:  # segment[DIRECTION] == DIAGONAL:
            print("What is doing a diagonal in here?")
        print(desc)
    print("The grid goes from ({},{}) to ({},{})".format(min_nums[X], min_nums[Y], max_nums[X], max_nums[Y]))

In [3]:
def get_processed_input(input_path, data, diagonals=False, extra_verbose=False):
    """A line segment is repersented by two tuples o 2 integers (coordinates). First one is the start and the second one is the end
    Returns the max. and min. numbers found to make easier building of the grid"""
    with open(input_path, "rt") as f:
        max_nums = (-inf, -inf)
        min_nums = (inf, inf)
        raw_input = f.read()
        raw_lines = raw_input.split("\n")
        for line in raw_lines:
            coords_raw = line.split(" -> ")
            coords = [c.split(",") for c in coords_raw]
            # Segment format (3-sized tuple) -> (([START][X],[START][Y]), ([END][X],[END][Y]), DIRECTION)
            segment = [(int(coords[START][X]),int(coords[START][Y])), ((int(coords[END][X])), int(coords[END][Y])), DIAGONAL]
            if segment[START][X] == segment[END][X]:
                segment[DIRECTION] = VERTICAL
            elif segment[START][Y] == segment[END][Y]:
                segment[DIRECTION] = HORIZONTAL
            if not diagonals and segment[DIRECTION] == DIAGONAL:
                if extra_verbose:
                    "({}, {})\t->\t({}, {}) discarded for being digonal".format(segment[START][X], segment[START][Y], segment[END][X], segment[END][Y])
                continue
            data.append(tuple(segment))
            max_nums = (max([max_nums[X], segment[START][X], segment[END][X]]), max([max_nums[Y], segment[START][Y], segment[END][Y]]))
            min_nums = (min([min_nums[X], segment[START][X], segment[END][X]]), min([min_nums[Y], segment[START][Y], segment[END][Y]]))
        print_data(data, 10, max_nums, min_nums)
        return min_nums, max_nums

In [11]:
def draw_grid(grid, separator="", zero_char="."):
    res = ""
    for y in range(len(grid[0])):
        for x in range(len(grid)):
            if grid[x][y]:
                res += str(grid[x][y])
            else:
                res += zero_char
            if x < len(grid[0]) - 1:
                res += separator
        if y < len(grid[0]) - 1:
            res += "\n"
    print(res)
        

In [13]:
def part_a(data, min_nums, max_nums, verbose=False):
    full_size = (max_nums[X] - min_nums[X], max_nums[Y] - min_nums[Y])
    grid = [[0 for i in range(full_size[X] +1)] for j in range(full_size[Y]+1)]
    if verbose:
        print("The size of the grid is {}x{} for a total of {} points" .format(full_size[X], full_size[Y], full_size[X] * full_size[Y]))
    offsets = (-min_nums[X], -min_nums[Y])  # If all the numbers are natural, these will be negative
    
    overlaped_points = set({})  # Probably the best way of avoiding duplicates
    max_overlaps = 0      # debug/curiosity
    max_overlaped = None  # debug/curiosity
    for segment in data:
        if segment[DIRECTION] == DIAGONAL:
            continue
        seg_mins = (min(segment[START][X], segment[END][X]), min(segment[START][Y], segment[END][Y]))
        seg_maxs = (max(segment[START][X], segment[END][X]), max(segment[START][Y], segment[END][Y]))
        for x in range(seg_mins[X], seg_maxs[X] + 1):
            for y in range(seg_mins[Y], seg_maxs[Y] + 1):
                grid_x = x+offsets[X]
                grid_y = y+offsets[Y]
                grid[grid_x][grid_y] += 1
                overlaps = grid[grid_x][grid_y]
                point = (x, y)
                if overlaps > 1:
                    overlaped_points.add(point)
                if overlaps > max_overlaps:
                    max_overlaps = overlaps
                    max_overlaped = point
    if verbose:
        draw_grid(grid, separator="  ")
    return len(overlaped_points), max_overlaped, max_overlaps

In [6]:
data = []
input_path = "input.txt"
min_nums, max_nums = get_processed_input(input_path, data, diagonals=False, extra_verbose=True)

First 10 (from a total of 338) line segments:
(510, 771)	->	(510, 322):	VERTICAL of size	449
(753, 99)	->	(753, 280):	VERTICAL of size	181
(160, 330)	->	(33, 330):	HORIZONTAL of size	127
(700, 793)	->	(700, 892):	VERTICAL of size	99
(327, 168)	->	(327, 690):	VERTICAL of size	522
(264, 203)	->	(264, 839):	VERTICAL of size	636
(135, 134)	->	(314, 134):	HORIZONTAL of size	179
(209, 759)	->	(41, 759):	HORIZONTAL of size	168
(466, 952)	->	(466, 135):	VERTICAL of size	817
(339, 828)	->	(339, 730):	VERTICAL of size	98
The grid goes from (12,11) to (987,986)


In [7]:
# There might be more points with the max. overlaps, but only the first one that reaches that value is shown
sol_a, max_overlaped, max_overlaps = part_a(data, min_nums, max_nums, verbose=False)
print("SOL A: [{}] points have at least two lines overlapping them, being ({}, {}) \
the most overlapped one with a total of {} lines".format(sol_a, max_overlaped[0], max_overlaped[1], max_overlaps))

SOL A: [6397] points have at least two lines overlapping them, being (773, 731) the most overlapped one with a total of 3 lines


## TEST

### A

Sol = 5

In [14]:
data_test = [
    ((0,9),(5,9),HORIZONTAL),
    ((8,0),(0,8),DIAGONAL),
    ((9,4),(3,4),HORIZONTAL),
    ((2,2),(2,1),VERTICAL),
    ((7,0),(7,4),VERTICAL),
    ((6,4),(2,0),DIAGONAL),
    ((0,9),(2,9),HORIZONTAL),
    ((3,4),(1,4),HORIZONTAL),
    ((0,0),(8,8),DIAGONAL),
    ((5,5),(8,2),DIAGONAL)
]
min_nums_test = (0,0)
max_nums_test = (9,9)
sol_test, max_overlaped_test, max_overlaps_test = part_a(data_test, min_nums_test, max_nums_test, verbose=True)
print("SOL A: [{}] points have at least two lines overlapping them, being ({}, {}) \
the most overlapped one with a total of {} lines".format(sol_test, max_overlaped_test[0], max_overlaped_test[1],
                                                         max_overlaps_test))

The size of the grid is 9x9 for a total of 81 points
.  .  .  .  .  .  .  1  .  .
.  .  1  .  .  .  .  1  .  .
.  .  1  .  .  .  .  1  .  .
.  .  .  .  .  .  .  1  .  .
.  1  1  2  1  1  1  2  1  1
.  .  .  .  .  .  .  .  .  .
.  .  .  .  .  .  .  .  .  .
.  .  .  .  .  .  .  .  .  .
.  .  .  .  .  .  .  .  .  .
2  2  2  1  1  1  .  .  .  .
SOL A: [5] points have at least two lines overlapping them, being (7, 4) the most overlapped one with a total of 2 lines
